# Object references, mutability and recycling

A name is not the object; a name is a separate thing.

If you imagine variables are like boxes, you can’t make sense of assignment in Python. Think of variables as Post-it notes. 

In [5]:
#Variables are assigned to objects only after the objects are created.

class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))
    
x = Gizmo()
#y doesn't get created, because the multiplication step takes place first
y = Gizmo() * 10
dir()

Gizmo id: 4404047096
Gizmo id: 4404044128


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [8]:
#an example of aliasing.
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
print(lewis is charles)
print(id(charles), id(lewis))

#alex is not an alias for charles: these variables are bound to distinct objects. The 
#objects bound to alex and charles have the same value — that’s what == compares — but 
#they have different identities.
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles
print(alex is not charles)

True
4403896112 4403896112
True


The == operator compares the values of objects (the data they hold), while is compares their identities. The is operator is faster than ==, because it cannot be overloaded, so Python does not have to find and invoke special methods to evaluate it, and computing is as simple as comparing two integer ids. 

## The relative immutability of tuples

The immutability of tuples really refers to the physical contents of the tuple data structure (ie. the references it holds), and does not extend to the referenced objects.

In [13]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1==t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1)
#The identity of list at t1[-1] has not changed, only its value.
print(id(t1[-1]))
#but t1 and t2 are now different, because t1[-1] has changed
print(t1==t2)

True
4403120072
(1, 2, [30, 40, 99])
4403120072
False
[3, [55, 44], (7, 8, 9)]
True
False


Using the constructor or [:] produces a shallow copy, i.e. the outermost container is duplicated, but the copy is filled with references to the same items held by the original container.

In [16]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
print(l2)
print(l1 == l2)
print(l1 is l2)

l1 = [3, [66, 55, 44], (7, 8, 9)] 
l2 = list(l1) # make a shallow copy - clone the top list, but not the contained list / tuple
l1.append(100) # append 100 to L1 - doesn't affect l2
l1[1].remove(55) # remove 55 from L1 list at pos. 2
print('l1:', l1)
print('l2:', l2) 
l2[1] += [33, 22] # For a mutable object like the list referred by l2[1], the operator += changes the list in-place.
l2[2] += (10, 11) # += on a tuple creates a new tuple and rebinds the variable l2[2]
print('l1:', l1) 
print('l2:', l2)

[3, [55, 44], (7, 8, 9)]
True
False
l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


## Deep copies

In [22]:
import copy

class Bus:
    def __init__(self, passengers=None): 
        if passengers is None:
            self.passengers = [] 
        else:
            self.passengers = list(passengers) 

    def pick(self, name):
        self.passengers.append(name) 

    def drop(self, name):
        self.passengers.remove(name)
    
bus1 = Bus(['Steve', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(id(bus1), id(bus2), id(bus3))
print(bus1.drop('Bill'))
print(bus2.passengers)
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
print(bus3.passengers)

4404044240 4404196128 4404196632
None
['Steve', 'Claire', 'David']
4403172680 4403172680 4404200008
['Steve', 'Bill', 'Claire', 'David']


## Function parameters as references

The only mode of parameter passing in Python is "call by sharing". Call by sharing means that each formal parameter of the function gets a copy of each reference in the arguments. In other words, the parameters inside the function become aliases of the actual arguments.

In [27]:
def f(a, b):
    a+=b
    return a

x = 1
y = 2
print(f(x,y))
#int x is unchanged
print(x,y)

a = [1,2]
b = [3,4]
print(f(a,b))
#list a is changed
print(a,b)

t = (10,20)
u = (30,40)
f(t,u)
#tuple t is unchanged
print(t,u)

3
1 2
[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]
(10, 20) (30, 40)


In [31]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    def __init__(self, passengers=[]): 
        self.passengers = passengers
    def pick(self, name): 
        self.passengers.append(name)
    def drop(self, name): 
        self.passengers.remove(name)

bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)
bus2 = HauntedBus()
bus2.pick('Carrie')
print(bus2.passengers)
bus3 = HauntedBus()
#bus3.passengers is not empty!
#The problem: bus2.passengers and bus3.passengers refer to the same list. 
#They were both instantiated with the same list
print(bus3.passengers)

['Alice', 'Bill']
['Bill', 'Charlie']
['Carrie']
['Carrie']


The issue with mutable defaults explains why None is often used as the default value for parameters that may receive mutable values. 

### Defensive programming with mutable parameters

In [34]:
class TwilightBus:
    """A bus model that makes passengers vanish"""
    def __init__(self, passengers=None): 
        if passengers is None:
            self.passengers = [] 
        else:
#             self.passengers = passengers
            #make a copy of the passengers list, rather than an alias
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name) 
    def drop(self, name):
        self.passengers.remove(name)

basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

bus = TwilightBus(basketball_team)

bus.drop('Tina')
bus.drop('Pat')
#When the methods .remove() and .append() are used 
#with self.passengers we are actually mutating the 
#original list received as argument to the constructor.
print(basketball_team)

['Sue', 'Tina', 'Maya', 'Diana', 'Pat']


## del and garbage collection

Objects are never explicitly destroyed; however, when they become unreachable they may be garbage-collected. The del statement deletes names, not objects. An object may be garbage collected as result of a del command, but only if the variable deleted holds the last reference to the object, or if the object becomes unreachable4. Rebinding a variable may also cause the number of references to an object reach zero, causing its destruction.

In [37]:
#del does not delete objects, but objects may be deleted as a 
#consequence of being unreachable after del is used.

import weakref

s1={1,2,3}
#s1 and s2 are aliases referring to the same set, {1, 2, 3}.
s2=s1

#This function must not be a bound method
def bye():
    print('Gone with the wind...')

#Register the bye callback on the object referred by s1.
ender = weakref.finalize(s1, bye) 
print(ender.alive)

del s1

print(ender.alive)

#Rebinding the last reference, s2, makes {1, 2, 3} unreachable. 
#It is destroyed, the bye callback is invoked and ender.alive becomes False.
s2 = 'spam'

print(ender.alive)

Gone with the wind...
True
True
Gone with the wind...
False


## Weak references

The presence of references is what keeps an object alive in memory. When the reference count of an object reaches zero, the garbage collector disposes of it. But sometimes it is useful to have a reference to an object that does not keep it around longer than necessary. A common use case is a cache.

-  Weak references to an object do not increase its reference count. The object that is the target of a reference is called the referent. Therefore, we say that a weak reference does not prevent the referent from being garbage collected.
-  Weak references are useful in caching applications because you don’t want the cached objects to be kept alive just because they are referenced by the cache.

In [39]:
import weakref

a_set = {0, 1}

wref = weakref.ref(a_set)

print(wref)
print(wref())

a_set = {2, 3, 4}

#doesn't work?
print(wref())
print(wref() is None)
print(wref() is None)

<weakref at 0x106824228; to 'set' at 0x1063e93c8>
{0, 1}
None
True
True


In [44]:
import weakref

class Cheese:
    def __init__(self, kind):
        self.kind = kind 
    def __repr__(self):
        return 'Cheese(%r)' % self.kind

#The stock maps the name of the cheese to a weak reference to the cheese instance
in the catalog.
#stock is a WeakValueDictionary
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese

print(sorted(stock.keys()))
del catalog
print(sorted(stock.keys()))
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
[]


### Limitations of weak references
Not every Python object may be the target, or referent, of a weak reference. Basic list and dict instances may not be referents, but a plain subclass of either can solve this problem easily:

In [45]:
t1=(1,2,3) 
t2 = tuple(t1)
#t1 and t2 are bound to the same object.
print(t2 is t1)
#And so is t3.
t3 = t1[:]
print(t3 is t1)


True
True
